# Pouvoir d’arrêt collisionnel des protons
## 1.
Typiquement, la plus grande énergie utilisée dans les centres de photonthérapie est de 230 MeV.

L'énergie totale d'un corps s'écrit $E=\gamma mc^2$ alors que l'énergie de repos s'écrit $E_0=mc^2$, donc l'énergie cinétique est $T=E-E_0=(\gamma-1)mc^2$. En inversant cette équation, on obtient $$\boxed{\gamma=\frac{T}{mc^2}+1}$$ Par leur définition, $\beta=\frac{v}{c}$ et $\gamma=\frac{1}{\sqrt{1-\frac{v^2}{c^2}}}$. Par substitution, on voit que $$\boxed{\gamma=\frac{1}{\sqrt{1-\beta^2}}}\quad\text{et}\quad\boxed{\beta=\sqrt{1-\frac{1}{\gamma^2}}}$$

## 2.
La densité électronique ($n_e$) est le nombre d'électrons par unité de volume, ce qui correspond à la somme des densités atomiques des différents éléments du matériau multipliées par leur nombre de proton ($Z$) par atome correspondant (en présumant que le matériau est électriquement neutre). La densité atomique d'un élément correspond à la densité massique du matériau ($\rho$) en grammes par cm$^3$ multipliée par la fraction de la masse du matériau appartenant à l'élément ($\xi$) divisée par la masse atomique de l'élément ($m_Z$) en grammes par mole, le tout multiplié par le nombre d'Avogadro ($N_o$) pour que les grammes par moles deviennent des grammes par atome. Autrement dit, $$n_e=\rho N_o\sum_{Z=1}^\infty\frac{Z\xi_Z}{m_Z}$$

In [14]:
def densite_elec_massique(masses_materiau, masses_atomiques):
    densite = 0
    for i in masses_materiau:
        densite += i * masses_materiau[i] / masses_atomiques[i]
    return densite


masses_atomiques = {
    1: 1.0079,
    6: 12.0107,
    7: 14.0067,
    8: 15.9994,
    12: 24.305,
    15: 30.9738,
    16: 32.065,
    20: 40.078,
    30: 65.39,
}
avogadro = 6.022 * 10 ** 23
masses_eau = {1: 0.111894, 8: 0.888106}
masses_os = {
    1: 0.047234,
    6: 0.144330,
    7: 0.041990,
    8: 0.446096,
    12: 0.002200,
    15: 0.104970,
    16: 0.003150,
    20: 0.209930,
    30: 0.000100
}
dens_masse_os = 1.85
dens_masse_eau = 1


densite_elec_eau = dens_masse_eau * avogadro * densite_elec_massique(masses_eau, masses_atomiques)
densite_elec_os = dens_masse_os * avogadro * densite_elec_massique(masses_os, masses_atomiques)


print("Densité électronique de l'eau: {0} électrons par cm^3\n".format(densite_elec_eau))
print("Densité électronique de l'os cortical: {0} électrons par cm^3".format(densite_elec_os))

Densité électronique de l'eau: 3.342731617093692e+23 électrons par cm^3

Densité électronique de l'os cortical: 5.8077060231519456e+23 électrons par cm^3


In [ ]:
import matplotlib as mp
import numpy as np


def pouvoir(energie, dens, re, me, c, Tmax):
    coefficient = 2 * np.pi * re ** 2 * me * c ** 2 * dens / 


re = 2.82 * 10 ** (-15)  # Rayon classique de l'électron (m)
me = 9.109 * 10 ** (-31)  # Masse classique de l'électron (kg)
c = 2.99792458 * 10 ** (8)  # Vitesse de la lumière (m/s)
Tmax = 230  # Énergie cinétique maximale (MeV)


energie_eau = 75
energie_os = 106.4


